In [177]:
import torch
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F
%matplotlib inline

In [221]:
pwd

'/Users/philippschott/Desktop/Python/NN-ben'

In [178]:
words = open('names.txt','r').read().splitlines()

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [204]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []

for w in words:
    #print (w)
    context = [0] * block_size
    for ch in (w + '.'):
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[i] for i in context), '—->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [205]:
g = torch.Generator().manual_seed(2147483647) 
# generator=g, requires_grad=True

#layer parameter
C = torch.randn((27,2), generator=g) #lookuptable
W1 = torch.randn((6,100), generator=g) #6 cause of 3x2 in the layer before
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)

parameters = [C, W1, W2, b1, b2]
for p in parameters: 
    p.requires_grad = True
    
sum(p.nelement() for p in parameters) 

3481

In [220]:
for _ in range(10):
    #forward pass
    emb = C[X] #Xx3x2
    h = torch.tanh(emb.view(emb.shape[0],6) @ W1 + b1) #merging second 2 dimensions together so Xx100
    logits = h @ W2 + b2 #Xx27

    #operations on the last layer
    '''counts = logits.exp()
    prob = counts / counts.sum(1, keepdims = True) #softmax
    loss = -prob[torch.arange(32),Y].log().mean()'''
    #same but simpler:
    loss = F.cross_entropy(logits, Y)

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    for p in parameters: 
        p.data += -0.1 * p.grad

    print(loss.item())

3.0437026023864746
3.0301942825317383
3.01741886138916
3.005323886871338
2.9938666820526123
2.983013868331909
2.9727377891540527
2.9630160331726074
2.9538276195526123
2.945152997970581
